# 퍼셉트론

In [48]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'1.15.0'

In [3]:
T=1.
F=0.
bias=1.

In [4]:
def get_AND_data():
    X = [
        [F, F, bias],
        [F, T, bias],
        [T, F, bias],
        [T, T, bias]
    ]
    Y = [[F], [F], [F], [T]]
    return X, Y

In [5]:
def get_OR_data():
    X = [
        [F, F, bias],
        [F, T, bias],
        [T, F, bias],
        [T, T, bias],
    ]
    Y = [[F], [T], [T], [T]]
    return X, Y

In [6]:
def get_XOR_data():
    X = [
        [F, F, bias],
        [F, T, bias],
        [T, F, bias],
        [T, T, bias]
    ]
    Y = [[F], [T], [T], [F]]
    return X, Y

* 학습 데이터

In [18]:
# X = 4 x 3
# Y = 4 x 1
X, Y = get_OR_data()

* 퍼셉트론(perceptron)

In [19]:
# W = 3 x 1
W = tf.Variable(tf.random_normal([3, 1]))

* 활성함수(activation function)는 스텝함수(step function)로 정함

In [20]:
def step(x):
    return tf.to_float(tf.greater(x, 0))

In [21]:
# X = 4 x 3
# W = 3 x 1
# f = 4 x 1
f = tf.matmul(X, W)
f

<tf.Tensor 'MatMul_2:0' shape=(4, 1) dtype=float32>

In [22]:
output = step(f)
output

<tf.Tensor 'ToFloat_1:0' shape=(4, 1) dtype=float32>

In [23]:
# 실제 값과 (퍼셉트론 -> 스텝함수) 출력 결과 비교
error = tf.subtract(Y, output)
error

<tf.Tensor 'Sub_1:0' shape=(4, 1) dtype=float32>

In [24]:
# 손실함수(loss function)
mse = tf.reduce_mean(tf.square(error))
mse

<tf.Tensor 'Mean_1:0' shape=() dtype=float32>

In [25]:
# 최적화
# X = 4 x 3
# erorr = 4 x 1
# transpose_a=True -> transpose X
delta = tf.matmul(X, error, transpose_a=True)
delta

<tf.Tensor 'MatMul_3:0' shape=(3, 1) dtype=float32>

In [26]:
# W = 3 x 1
# tf.add(W, delta) = 3 x 1
# W 의 값을 업데이트
# 즉 W = W + delta
train = tf.assign(W, tf.add(W, delta))
train

<tf.Tensor 'Assign_1:0' shape=(3, 1) dtype=float32_ref>

In [27]:
init = tf.global_variables_initializer()

In [28]:
with tf.Session() as sess:
    sess.run(init)
    err = 1
    epoch, max_epochs = 0, 20
    while err > 0. and epoch < max_epochs:
        epoch += 1
        err = sess.run(mse)
        sess.run(train)
        print('epoch', epoch, 'mse', err)
        
    print('\nTesting Result\n', sess.run([output])[0].T)

epoch 1 mse 0.75
epoch 2 mse 0.25
epoch 3 mse 0.25
epoch 4 mse 0.25
epoch 5 mse 0.25
epoch 6 mse 0.0

Testing Result
 [[0. 1. 1. 1.]]


# 다층 퍼셉트론

* 데이터 텐서가 들어올 입구

In [32]:
X = tf.placeholder(tf.float32, shape=[4, 2])
Y = tf.placeholder(tf.float32, shape=[4, 1])

* 첫째 레이어

In [33]:
W1 = tf.Variable(tf.random_uniform([2, 2]))
W1

<tf.Variable 'Variable_3:0' shape=(2, 2) dtype=float32_ref>

In [35]:
B1 = tf.Variable(tf.zeros([2]))
B1

<tf.Variable 'Variable_5:0' shape=(2,) dtype=float32_ref>

In [36]:
# X = 4 x 2
# W1 = 2 x 2
# Z = 4 x 2
Z = tf.tanh(tf.matmul(X, W1) + B1)
Z

<tf.Tensor 'Tanh:0' shape=(4, 2) dtype=float32>

* 둘째 레이어

In [37]:
W2 = tf.Variable(tf.random_uniform([2, 1]))
W2

<tf.Variable 'Variable_6:0' shape=(2, 1) dtype=float32_ref>

In [38]:
B2 = tf.Variable(tf.zeros([1]))
B2

<tf.Variable 'Variable_7:0' shape=(1,) dtype=float32_ref>

* 모델 예측 결과가 나올 통로

In [39]:
# Z = 4 x 2
# W2 = 2 x 1
# Y_hat = 4 x 1
Y_hat = tf.tanh(tf.matmul(Z, W2) + B2)
Y_hat

<tf.Tensor 'Tanh_1:0' shape=(4, 1) dtype=float32>

* 손실함수

In [40]:
error = tf.subtract(Y, Y_hat)
error

<tf.Tensor 'Sub_2:0' shape=(4, 1) dtype=float32>

In [41]:
loss = tf.reduce_mean(tf.square(error))
loss

<tf.Tensor 'Mean_2:0' shape=() dtype=float32>

* 최적화

In [43]:
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [44]:
train_X, train_Y = get_XOR_data()

In [50]:
train_X = np.array(train_X)[:, :2]

In [52]:
init = tf.global_variables_initializer()

In [59]:
%%time
with tf.Session() as sess:
    sess.run(init)
    print("train data: ", train_X,'\n')
    
    for i in range(20000):
        sess.run(train, feed_dict={X: train_X, Y: train_Y})
        if i% 5000 == 0:
            print("Epoch:", i)
            print("Output: ", sess.run(Y_hat, feed_dict={X: train_X, Y: train_Y}))
            print()
    
    print("Final Output:", sess.run(Y_hat, feed_dict={X: train_X, Y: train_Y}))

train data:  [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]] 

Epoch: 0
Output:  [[4.0939497e-04]
 [7.7211833e-01]
 [7.8359747e-01]
 [8.9360303e-01]]

Epoch: 5000
Output:  [[0.048753  ]
 [0.64881986]
 [0.64895046]
 [0.6890365 ]]

Epoch: 10000
Output:  [[0.02412656]
 [0.6565876 ]
 [0.65650403]
 [0.6772099 ]]

Epoch: 15000
Output:  [[0.01518219]
 [0.65897   ]
 [0.6587086 ]
 [0.67003053]]

Final Output: [[0.01254397]
 [0.6580558 ]
 [0.65712637]
 [0.65895057]]
CPU times: user 24.9 s, sys: 2.33 s, total: 27.3 s
Wall time: 17 s
